# Main

This will be the notebook to perform the steps required to generate the topographical hexagons I desire. I will import the functions written in the other files and combine them in such a way as to create the hexagons I desire.

### Initialisation
This code will set up the environment required to produce the plots I would like. As such, I'll be importing my python scripts and any other required modules.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import loadData as LD
import CoordinateTransform as CT
import Hexagon as Hexagon

### Loading data
I will load in the data for both of the SRTM datasets and concatenate them into a single matrix.
From there, I will downsample the matrix twice until it is 1500 * 3000

In [2]:
seaVal = -2

D1,m1 = LD.load_asc_format('data\\srtm_35_04\\','srtm_35_04.asc',seaVal)
D2,m2 = LD.load_asc_format('data\\srtm_36_04\\','srtm_36_04.asc',seaVal)

D_combined = np.hstack((D1,D2))

In [ ]:
plt.figure(figsize = (10,4))

plt.subplot(1,4,1)
plt.imshow(D1,origin='lower',interpolation='bilinear')

plt.subplot(1,4,2)
plt.imshow(D2,origin='lower',interpolation='bilinear')

plt.subplot(1,2,2)
plt.imshow(D_combined,origin='lower',interpolation='bilinear')

plt.show()

### Downsampling and surface plotting

I will downsample the data to be able to create a surface plot. I will then be able to apply scaling functions to the downsampled data and see how they affect the model.

In [ ]:
# performing two downsamples on the matrix

newD,__ = LD.downsample_minimum(D_combined)
newD,__ = LD.downsample_minimum(newD)

In [ ]:
%matplotlib widget

fig = plt.figure(figsize = (8,4))
ax1 = fig.add_subplot(121, projection='3d')
x,y = range(np.size(newD,1)), range(np.size(newD,0))
X,Y = np.meshgrid(x,y)
ax1.plot_surface(X,Y,newD,rcount=100,ccount=100)

################################################################
idf = lambda x: x # identity function, for copying newD
nnewD = idf(newD)

#rescaling the heightmap by a function to exagerate lower elevations more
#k = 10
#w = lambda x: np.log(x)*k
#w = lambda x: np.sqrt(x)*k
#newD[newD > 0] = w(newD[newD > 0])

#w = lambda x: x/20
#newD[newD > 0] = w(newD[newD > 0])
w = lambda x: np.sqrt(x) #if x>0 else x
nnewD[nnewD > 0] = w(nnewD[nnewD > 0])

#################################################################

ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(X,Y,nnewD,rcount=100,ccount=100)

ax1.azim = 121
ax1.elev = 25
ax1.set_box_aspect((4,2,2))
ax2.set_zlim3d(seaVal-2)
ax2.azim = 121
ax2.elev = 25
ax2.set_box_aspect((4,2,2))
ax2.set_zlim3d(seaVal-2)
plt.show()

### Hexagons and culling

I will use the Hexagon and CoordinateTransform modules to cull our map in the shape of a hexagon with arbitrary centre and rotation.

In [ ]:
# start by creating the coordinate transform based on the metadata from the SRTM data.
newD,lcd = LD.downsample_minimum(D_combined)
#newM1 = m1
#newM1[2] = newM1[2] * lcd
ref1Img = [0,0]
ref1Coords = [m1[0],m1[1]]
ref2Img = [6000,3000]
ref2Coords = [0,45]
Transform = CT.CoordinateTransform(ref1Img,ref1Coords,None,ref2Img,ref2Coords)

# going to get the coordinate reference for Oviedo
Oviedo_coords = [-8.849259, 42.361374] #long,lat
Oviedo_img = Transform.coords2Img(Oviedo_coords)
print('Oviedo_img: {}'.format(Oviedo_img))

# create the hexagon of 200 pixels radius, centered on Oviedo
H = Hexagon.Hexagon(500,15,Oviedo_img)

# create all the points in image space
x = range(0,np.size(newD,1))
y = range(0,np.size(newD,0))
X,Y = np.meshgrid(x,y)
print(np.shape(X))

cullMask = H.outsideHexagon(X,Y)

plt.figure(figsize = (10,4))

plt.subplot(1,4,1)
plt.imshow(newD,origin='lower',interpolation='bilinear')


newD[cullMask] = -40

plt.subplot(1,4,2)
plt.imshow(newD,origin='lower',interpolation='bilinear')

H2 = H.createAdjacentHexagon()
newCullMask = H2.outsideHexagon(X,Y)
newD,__ = LD.downsample_minimum(D_combined)
newD[newCullMask] = -40
plt.subplot(1,4,3)
plt.imshow(newD,origin='lower',interpolation='bilinear')

newD,__ = LD.downsample_minimum(D_combined)
newD[np.logical_and(cullMask,newCullMask)] = -40
plt.subplot(1,4,4)
plt.imshow(newD,origin='lower',interpolation='bilinear')


newD created, size is (6000,3000)
Oviedo_img: [ 690.4446 1416.8244]
(6000, 3000)
newD created, size is (6000,3000)
newD created, size is (6000,3000)
